# HARD NEGATIVE PROMPT

In [1]:
from gen_social_instruction import get_prompt
import requests
import os
import time
import random
import json
import ast
import re
import pandas as pd

df = pd.read_csv('raw_data/wikipedia_vi/wikipedia_vi_cleaned_langdet.csv',index_col=[0])
all_text = df['text_cutted'].tolist()
len(all_text)

PATH = "./gpt4_generated_data/240202-wikipedia_vi_cleaned-idx0-thuc_prompt_v8"
LIST_FILE = os.listdir(PATH) + os.listdir("./gpt4_generated_data/240115-wikipedia_vi_cleaned-idx0-thuc_prompt_v8")

rep_lst=["\n","\"jsonCopy code", "lessCopy code", "Copy code\"}"]
rep = {i: "" for i in rep_lst} 
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))

if not os.path.isdir(PATH):
    os.makedirs(PATH)

def call_chatgpt(idx, post):
    if f'{idx}.json' not in LIST_FILE:
        try:
    #         print(f'{idx}.json: Not Available\n')
            time.sleep(random.randint(7, 17))
            url = "http://127.0.0.1:8081/send_prompt"

            prompt = get_prompt(post)

            payload = {'prompt': prompt, 'outerHTML': '0'}
            files = []
            headers = {}
            dict_iio = {}
            response = requests.request("POST", url, headers=headers, data=payload, files=files)
            output_raw_text = response.text
            dict_iio = {}

            data_path = f'{PATH}/{idx}.json'

            try:
                output_raw_text = output_raw_text.encode('utf-8').decode('unicode_escape')
                output_raw = output_raw_text.encode('utf-16', 'surrogatepass').decode('utf-16')
            except UnicodeEncodeError as e:
                print(f"Error encoding text: {e}")
                output_raw = output_raw_text

            output_raw = output_raw[output_raw.rfind("Danh sách câu hỏi không có câu trả lời")-1 : output_raw.rfind("]\n}")+1]
            processes_output = '{' + pattern.sub(lambda m: rep[re.escape(m.group(0))], output_raw) + '}'
    #         print(processes_output)

            try:
                dict_iio['instruction'] = prompt
                if processes_output != '{}':
                    dict_iio['output'] = ast.literal_eval(processes_output)
                    print("=" * 30)
                    print("idx: ", idx, "\nstatus: True")
                    print(processes_output[100:130])
            #         print(data_path)
                    with open(data_path, 'w', encoding='utf-8') as json_file:
                        json.dump(dict_iio, json_file, ensure_ascii=False, indent=4)
                        json_file.close()
            except:
                print("=" * 30)
                print("idx: ", idx, "\nstatus: False")
                pass
            time.sleep(random.randint(5, 10))

        except requests.RequestException as e:
            print(f"Error making request: {e}")
        except IOError as e:
            print(f"Error writing to file: {e}")

        return output_raw_text
    else:
        return None

In [ ]:
import concurrent.futures

start_idx = 10000
with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
#     future_to_url = {executor.submit(call_chatgpt, idx, post): (idx, post) for idx, post in enumerate(all_posts[start_idx:])}
    # Create a dictionary to track futures
    future_to_url = {executor.submit(call_chatgpt, start_idx + idx, post): (start_idx + idx, post) 
                     for idx, post in enumerate(all_text[start_idx:])}
    
    for future in concurrent.futures.as_completed(future_to_url):
        res = future.result(timeout=5000)
        time.sleep(random.randint(15, 30))

idx:  17202 
status: True
ềm tự do đã ảnh hưởng đến phát
idx:  17396 
status: True
"Quần thể chó rừng lưng đen hi
idx:  17373 
status: True
ớc khi trở thành tướng lãnh là
idx:  17245 
status: True
iện thuận lợi cho việc phát tr
idx:  17471 
status: True
như thế nào?",    "Lợi ích của
idx:  17337 
status: True
trẻ mồ côi ở Việt Nam hiện tại
idx:  17449 
status: True
c nào trong sự nghiệp của mình
idx:  17508 
status: True
ng ngoại bang trong suốt thời 
idx:  17523 
status: True
ng giám sát trước khi chuyến b
idx:  17534 
status: True
 tạp trong khoai tây chiên là 
idx:  17577 
status: True
cho chuyến đi của mình?",    "
idx:  17551 
status: True
hặn thảm họa tương tự như Fuku
idx:  17575 
status: True
ế nào trong việc duy trì đa dạ
idx:  17616 
status: True
êu?",    "Ai là thiết kế trưởn
idx:  17614 
status: True
 trình thiết kế và chế tạo tàu
idx:  17604 
status: True
ự nghiệp của mình?",    "Làm t
idx:  17610 
status: True
u thuộc lớp La Galissonnière l
idx:  17629 
status: True
ác tr

In [ ]:
## import random
# index = random.randint(7200, len(all_text))
# print(f"""
# Num of rows: {len(all_text)}
# INDEX: {index}
# CONTENT:\n{get_prompt(all_text[index])}
# """)

# # import random
# i = random.randint(30000, len(all_text))
# print(i)
# output = call_chatgpt(i, all_text[i])
# output

In [1]:
# from pathlib import Path
# import pandas as pd

# df1 = pd.read_csv('raw_data/wikipedia_vi/train-00000-of-00003-6218d2963e302058.csv', index_col=[0])
# df2 = pd.read_csv('raw_data/wikipedia_vi/train-00001-of-00003-12e6c4fadbec91d4.csv', index_col=[0])
# df3 = pd.read_csv('raw_data/wikipedia_vi/train-00002-of-00003-175fcfe1c45b0b85.csv', index_col=[0])
# df1['wordcount'] = [len(x.split()) for x in df1['text'].tolist()]
# df2['wordcount'] = [len(x.split()) for x in df2['text'].tolist()]
# df3['wordcount'] = [len(x.split()) for x in df3['text'].tolist()]
# full_df = pd.concat([df1, df2, df3])
# full_df.to_csv('raw_data/wikipedia_vi/wikipedia_vi.csv')

# import pandas as pd
# df = pd.read_csv('raw_data/wikipedia_vi/wikipedia_vi.csv',index_col=[0])
# df_filter = df[df['wordcount'] >= 512]
# df_filter['text_cutted'] = df_filter['text'].apply(lambda x: ' '.join(x.split()[:512]))
# df_filter['wordcount_cutted'] = [len(x.split()) for x in df_filter['text_cutted'].tolist()]
# df_filter.to_csv('raw_data/wikipedia_vi/wikipedia_vi_cleaned.csv')

# from ftlangdetect import detect

# df['lang'] = df['title'].apply(lambda x: detect(text=x, low_memory=False)['lang'])
# df = df[df['lang'] == 'vi']
# df['text_cutted'] = df.apply(lambda row: row['title'] + ' ' + row['text_cutted'] if not row['text_cutted'].startswith(row['title'].split()[0]) else row['text_cutted'], axis=1)

# df.to_csv('raw_data/wikipedia_vi/wikipedia_vi_cleaned_langdet.csv')

In [2]:
# {"accounts": [{"name": "Nova001", "email": "novachatbot001@gmail.com", "passw": "aia_@123", "idx": 0, "cookies": "novachatbot001@gmail.com.json"}, {"name": "Unicorn401", "email": "Unicorn401@68ww.onmicrosoft.com", "passw": "6WGQwwV9ZUmOMQu", "idx": 0, "cookies": "Unicorn401@68ww.onmicrosoft.com.json"}, {"name": "thegiknet1", "email": "thegiknet1@unicornshop.vn", "passw": "AH7F7QM83YNn59V", "idx": 0, "cookies": "thegiknet1@unicornshop.vn.json"}, {"name": "Centrix63", "email": "Centrix63@unicornshop.vn", "passw": "k38vo4xvDpyAaNu", "idx": 0, "cookies": "Centrix63@unicornshop.vn.json"}, {"name": "ChatGPT86", "email": "ChatGPT86@23ww.onmicrosoft.com", "passw": "5L4MAfmCQbd5VV0", "idx": 0, "cookies": "ChatGPT86@23ww.onmicrosoft.com.json"}, {"name": "ChatGPT87", "email": "ChatGPT87@23ww.onmicrosoft.com", "passw": "FYlkgdDGNSooStR", "idx": 0, "cookies": "ChatGPT87@23ww.onmicrosoft.com.json"}, {"name": "eggleston212", "email": "eggleston212@outlook.lv", "passw": "HappyNewYear2024", "idx": 0, "cookies": "eggleston212@outlook.lv.json"}, {"name": "thanhtungtruong2611", "email": "thanhtungtruong2611@gmail.com", "passw": "tungtruong2233@@", "idx": 0, "cookies": ""}]}

In [9]:
# Time: 18:49 (15/01)
# File: 436

# Time: 23:49 (15/01)
# File: 700

# Time: 13:35 (16/01)
# File: 2203

# Time: 19:00 (16/01)
# File: 2700

# Time: 06:30 (17/01)
# File: 4238

# Time: 10:13 (17/01)
# File: 4639
# Note: Acc 90k BAN (keep GPT Plus (40mess), BAN GPT Team (100mess))

# Time: 15:144 (18/01)
# File: 6598

# Time: 18:34 (19/01)
# File: 8691

# Time: 18:12 (29/01)
# File: 11500

# ===============================================

# Time: 15:51 (30/01)
# File: 12000

# Time: 14:21 (30/01)
# File: 12574

# Time: 18:04 (31/01)
# File: 12744